# Chapter 2

## Distributed computing at the command line

Paul E. Anderson

## Ice Breaker

What's is the best pizza place in SLO?

While this text can be viewed as PDF, it is most useful to have a Jupyter environment. I have an environment ready for each of you, but you can get your own local environment going in several ways. One popular way is with Anaconda (<a href="https://www.anaconda.com/">https://www.anaconda.com/</a>. Because of the limited time, you can use my server.

In [15]:
%load_ext autoreload
%autoreload 2


# Put all your solutions into Lab1_helper.py as this script which is autograded
import Chapter2_helper 

from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. 
# This is not relevant to most people because I recommended you use my server, but
# change home to where you are storing everything. Again. Not recommended.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Command Line

In Chapter 1, I included several videos where you had to follow along as I entered commands on the command line. In this chapter we are going to increase our mastery of the command line and distributed computing at the same time. 

The command line is very agile. It has a read-eval-print loop that lends itself to interactive exploration as well as automation. It has been a mainstay in the toolkit of modern computer science, and it will remain so for a long time to come.

The command line augments and amplifies technology we are already using. We'll use Spark, Hadoop, MongoDB, and other programs in this class. The command line is important across these technologies. 

The command line is scalable which is very important for distributed computing. It allows us to automate many tasks and scale them in a distributed manner.

The command line is ubiquitous. 95% of the worlds supercomputers use Unix/Linux and are accessible by the command line. 

### How are distributed systems different?
While we can abstract away some of elements of distributed computing, we are going to study approaches for:
1. How to store data on multiple systems?
2. How to handle updates and fix (or handle) inconsistencies?
3. How do we assemble the full answer?

## Practical Considerations
Most real world examples that need distributed computing need distributed computing because they would otherwise (and may still) require a long time to run. This isn't practical for learning. More importantly to remember, even in the real world we test on small subsets of data before scaling up. All of the examples throughout are scaled down representations of a real problem that may require distributed computing depending on time and resources.

## Parallel Processing Logs
Consider the cybersecurity task of examining web server logs. Specifically, read/skim this article:
<a href="https://resources.infosecinstitute.com/topic/log-analysis-web-attacks-beginners-guide/">Log Analysis</a>.

Once you have read the article, consider the problem of running Scalp on a single log. Probably not an issue. Consider running it on a very large log (web server logs can get very very big). 

Because they can get big they are often archived routinely. So now you have a problem of looking through many different log files. We may want to run Scalp on the logs over a long period of time. We may want to run it with different parameters. We may make mistakes and need to run it again quickly. You are starting to get my point I think.

Would this problem need distributed computing. Discuss in your groups and bring the answer back in a few minutes.

**It boils down to these questions for us over and over again (some of these overlap):**
* Do I need a more efficient (i.e., faster) approach?
* Can my task be broken down into components that may be analyzed and then combined?
* Is it worth the effort? 
* Can you just wait for this answer?
* Can you estimate how long a non-distributed approach will take?

## Detour: Linux and Bash

While there are many tutorials and introduction to Bash, I like this one: https://ubuntu.com/tutorials/command-line-for-beginners. You may do almost the entire tutorial directly in this notebook. There are several ways to run Bash within Jupyter. Here are some examples.

## Example Data: Project Gutenberg

Consider another example. What if you were interested in looking for patterns in the top 100 books last year? Your first idea is to compare the word frequencies in the top 25 books to the next 25 books. 
* You have a level of programming skill that makes writing a Python program to look through a single book (text file) within range. 
* You are familiar with Python dictionaries and can sequentially process a book. 

Let's design such a program together!

There is a list of books downloaded from Project Gutenberg in the data folder
* Website has over 60,000 books. 
* I downloaded the most popular books on 1/12/2021 
* The order in which they are ranked is in order.txt. 

Let's see a few Bash commands to take a look at the books.

In [16]:
!ls {home}/csc-369-student/data/gutenberg

1080-0.txt  158-0.txt	 2542-0.txt   41-0.txt	     6130-0.txt   84-0.txt
11-0.txt    160-0.txt	 2554-0.txt   42108-0.txt    6133-0.txt   863-0.txt
113-0.txt   16-0.txt	 2591-0.txt   4300-0.txt     64238-0.txt  902-0.txt
1184-0.txt  1661-0.txt	 25929-0.txt  43-0.txt	     64239-0.txt  98-0.txt
120-0.txt   1727-0.txt	 2600-0.txt   45-0.txt	     64241-0.txt  996-0.txt
1232-0.txt  1952-0.txt	 2701-0.txt   4517-0.txt     64242-0.txt  group1
1250-0.txt  1998-0.txt	 28054-0.txt  46-0.txt	     64244-0.txt  group2
1260-0.txt  203-0.txt	 2814-0.txt   50040-0.txt    64246-0.txt  group3
1342-0.txt  205-0.txt	 2852-0.txt   50040-0.txt.1  64247-0.txt  order.txt
135-0.txt   209-0.txt	 30254-0.txt  521-0.txt      730-0.txt
1399-0.txt  215-0.txt	 35-0.txt     53854-0.txt    74-0.txt
1400-0.txt  219-0.txt	 3600-0.txt   57426-0.txt    76-0.txt
140-0.txt   244-0.txt	 36-0.txt     58585-0.txt    766-0.txt
147-0.txt   25344-0.txt  408-0.txt    60479-0.txt    768-0.txt


In [17]:
!ls -l {home}/csc-369-student/data/gutenberg | wc -l

80


In [18]:
!head {home}/csc-369-student/data/gutenberg/order.txt

50040
1342
84
6133
46
11
1080
1661
98
25344


**Python code to create a list of files in the ranked order**

In [19]:
from os import path
book_files = []
for book in open(f"{home}/csc-369-student/data/gutenberg/order.txt").read().split("\n"):
    if path.isfile(f'{home}/csc-369-student/data/gutenberg/{book}-0.txt'):
        book_files.append(f'{home}/csc-369-student/data/gutenberg/{book}-0.txt')
book_files[:10]

['/home/jupyter-pander14/csc-369-student/data/gutenberg/50040-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/1342-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/84-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/6133-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/46-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/11-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/1080-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/1661-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/98-0.txt',
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/25344-0.txt']

### What is our top book?

In [20]:
!head {book_files[0]}

### What is our second book?

In [21]:
!head {book_files[1]}

**Exercise 1:** Write a function that counts the number of times a word appears in a book. In other words, we want to know the frequencies of the words in each book. One way to store this is in a dictionary for each book. Do not worry about punctuation or capitalization. This is just an exercise.

In [22]:
book_word_freq = Chapter2_helper.count_words(book_files)
import pandas as pd
pd.Series(book_word_freq)

/home/jupyter-pander14/csc-369-student/data/gutenberg/50040-0.txt    {'﻿': 1, 'St.': 6, 'Benedict’s': 2, 'Rule': 22...
/home/jupyter-pander14/csc-369-student/data/gutenberg/1342-0.txt     {'﻿': 1, 'The': 271, 'Project': 78, 'Gutenberg...
/home/jupyter-pander14/csc-369-student/data/gutenberg/84-0.txt       {'﻿Project': 1, 'Gutenberg's': 1, 'Frankenstei...
/home/jupyter-pander14/csc-369-student/data/gutenberg/6133-0.txt     {'﻿The': 1, 'Project': 78, 'Gutenberg': 21, 'E...
/home/jupyter-pander14/csc-369-student/data/gutenberg/46-0.txt       {'﻿The': 1, 'Project': 78, 'Gutenberg': 21, 'E...
                                                                                           ...                        
/home/jupyter-pander14/csc-369-student/data/gutenberg/730-0.txt      {'﻿The': 1, 'Project': 79, 'Gutenberg': 22, 'E...
/home/jupyter-pander14/csc-369-student/data/gutenberg/113-0.txt      {'﻿': 1, 'The': 219, 'Project': 79, 'Gutenberg...
/home/jupyter-pander14/csc-369-student/data/gute

**If you want to time it, there is a magic command for that.**

In [29]:
%%timeit -n 1
book_word_freq = Chapter2_helper.count_words(book_files)

10.6 s ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Motivating Example: Inverted Index

Information retrieval (IR) is a field concerned with the structure, analysis, organization, storage, searching, and retrieval of information.

IR is the science of searching for documents, information within documents, and metadata about documents. Documents as used here describe an abstract piece of information that includes databases and the World Wide Web (WWW).

**Stop and think:** Take a moment and a piece of paper. Sketch out how you think the backend and the frontend process/architecture of search engine is constructed. Very high level of course. 

Image commented out by default.

<!--<img src="https://www.researchgate.net/profile/Yoshiki-Mikami/publication/224197661/figure/fig1/AS:341147890274356@1458347403868/Architecture-of-Search-Engine.png" width=400>-->

We will not dig into all of the aspects shown in that image. 

We will focus on one important algorithm: the creation of the inverted index. 

We will implement this algorithm and apply it to produce an inverted index of our Project Gutenberg Dataset.

<img src="https://hdscorp--c.na74.content.force.com/servlet/servlet.ImageServer?id=0151J000004MM9EQAW&oid=00Do0000000IJig">

**Stop and think:** How could it help someone trying to retrieve information from Project Gutenberg?

It provides a quick and efficient data structure to find the location of words/terms in our dataset. We could imagine a situation where an author or scholar would want to look up all occurences of a word, and it is reasonable to assume that they would benefit from viewing the nearby text containing the word/term.

### Implementation thoughts

We will write algorithms throughout this course using different technology. 

Our first approach to creating an inverted index using distributed computing will rely on minimal technology. Specifically, we will rely on our knowledge of Python and the command line.

Later in the class, we will repeat this example using Hadoop and Spark.

Before you stop drop and code, please consider the following questions:

1. What part of computing the inverted index could be distributed? Think in terms of subtasks that can run in parallel.

2. Once you have the results of these subtasks (i.e., the parallel processing finishes), how do you combine the results into a single inverted index?

## GNU Parallel
One of my favorite command line finds of all time: <a href="https://www.gnu.org/software/parallel/">https://www.gnu.org/software/parallel/</a>. This flexible program provides an easy to use way of running a command in *parrallel*. We will illustrate with a silly example, but it illustrates the point of *parallel*. 

Let's say you want to list the contents of three directors: dir1, dir2, and dir3. You can do this with:

```bash
$ ls dir1

$ ls dir2

$ ls dir3
```

Using parallel you can do:

```bash
parallel ls {} ::: dir1 dir2 dir3
```

This command can be brocken down into the list after ::: that is split and inserted into {}. Each of these commands are excuted in parallel now. 

**Stop and think:** Do you know which directory will be listed first?

Consider creating an inverted index for our small example of <100 books. 

Technically, our small example wouldn't classify as something that needs distributed computing, but consider computing the architecture of a search engine we described previously. Creating an inverted index for the WWW is an application that needs distributed computing. 

In both scenarios, the algorithms and processes are conceptually similar.

We will be leaving the implementation of an inverted index for Project Gutenberg to a lab exercise.

For this lecture, we will discuss and demonstrate *parallel* by constructing a distributed computing version of our word/term frequency algorithm.

The **strength** of the GNU parallel project is it seamlessly takes something written (e.g., program, script) and executes it in parallel.

The burden of constructing a merged/final output is not covered by parallel and must be done by you. More sophisticated technologies such as Spark provide built-in support for this task.

Still, we all have many useful programs not optimized for distributed computing that nevertheless could benefit from distributed computing.

## Case study: Term Frequency

### Running parallel

We will break down a more sophisticated *parallel* command and answer the following:
* Identify the pipes. What are they doing?
* What is the structure of the find command?
* What happened to the ::: in the parallel command
* What does the -v mean in the egrep command?

In [9]:
!find ~/csc-369-student/data/gutenberg -name "*.txt" | egrep -v order.txt | parallel echo {}

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence this citation notice: run 'parallel --citation'.

/home/jupyter-pander14/csc-369-student/data/gutenberg/1232-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/45-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/6133-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/2814-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/46-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/30254-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/3600-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/219-0.tx

/home/jupyter-pander14/csc-369-student/data/gutenberg/768-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/2554-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/120-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/2542-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/1260-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/36-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/35-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/2701-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/64241-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/64244-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/1400-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/135-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/158-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/209-0.txt
/home/jupyter-pander14/csc-369-student/data/gutenberg/160-0.txt
/home/jupyter-pander14/csc-369-st

Often I use this pattern where echo is the last command. This helps me debug before I even get started. To me programming is about debugging more than anything. The better I am at debugging, the better programmer. 

Next we will use a script ``Chapter2_count_words_book.py`` and *parallel* to perform a distributed computation of counting words in parallel.

In [30]:
%%timeit -n 1
!find ~/csc-369-student/data/gutenberg -name "*.txt" | egrep -v order.txt | parallel python Chapter2_count_words_book.py

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence this citation notice: run 'parallel --citation'.

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence this citation notice: run 'parallel --citation'.

Academic tradition requires 

### Results
* We got a speedup even though we had to start Python multiple times (our results may vary on a shared environment). 
* There is always overhead when moving towards distributed computing. 
* We aren't going to do the actual comparison of top 25 to next 25. 
* Well... why not. We are almost there. This material is definitely not part of this class though.

In [14]:
import pandas as pd
import altair as alt

book_word_freq = Chapter2_helper.count_words(book_files)

top_books = book_files[:25]
next_books = book_files[25:50]

top_df = pd.DataFrame(columns=["book","word","freq","group"]).set_index(["book","word"])
next_df = pd.DataFrame(columns=["book","word","freq","group"]).set_index(["book","word"])

# Normalize the counts for each book
for book in top_books:
    data = pd.Series(book_word_freq[book])
    data = (data/data.sum()).to_frame().reset_index()
    data.columns=["word","freq"]
    data["book"] = book
    data["group"] = "top"
    top_df = top_df.append(data.set_index(["book","word"]))
    
for book in next_books:
    data = pd.Series(book_word_freq[book])
    data = (data/data.sum()).to_frame().reset_index()
    data.columns=["word","freq"]
    data["book"] = book
    data["group"] = "bottom"
    next_df = next_df.append(data.set_index(["book","word"]))

In [15]:
next_df = next_df.reset_index()
top_df = top_df.reset_index()

In [16]:
plot_df = top_df.append(next_df)
plot_df

,book,word,freq,group
0,/home/jupyter-pander14/csc-369-student/data/gu...,﻿,0.000040,top
1,/home/jupyter-pander14/csc-369-student/data/gu...,St.,0.000238,top
2,/home/jupyter-pander14/csc-369-student/data/gu...,Benedict’s,0.000079,top
3,/home/jupyter-pander14/csc-369-student/data/gu...,Rule,0.000872,top
4,/home/jupyter-pander14/csc-369-student/data/gu...,for,0.009237,top
...,...,...,...,...
496984,/home/jupyter-pander14/csc-369-student/data/gu...,produce,0.000015,bottom
496985,/home/jupyter-pander14/csc-369-student/data/gu...,subscribe,0.000015,bottom
496986,/home/jupyter-pander14/csc-369-student/data/gu...,email,0.000015,bottom
496987,/home/jupyter-pander14/csc-369-student/data/gu...,newsletter,0.000015,bottom


In [17]:
pivot_df = plot_df.groupby(['word','group']).mean().reset_index().pivot_table(index='group',columns='word').T

In [18]:
top_k = 30
top_words = [v[1] for v in (pivot_df["bottom"] - pivot_df["top"]).abs().sort_values(ascending=False)[:top_k].index]
top_words

['MRS',
 'I',
 'Scott',
 'Hester',
 'of',
 'he',
 'Alice',
 'and',
 'the',
 'Martians',
 'hee',
 'Abbot',
 'vnto',
 'Quixote,',
 '"I',
 'Chap.',
 'Tom',
 'Harriet',
 'Weston',
 'his',
 'Office',
 'Jennie',
 'my',
 'Elizabeth',
 'Jekyll',
 'or',
 'Jim',
 'Anne.',
 'Anne',
 'PART']

In [19]:
alt.Chart(plot_df.set_index('word').loc[top_words].reset_index()).mark_bar().encode(
    x='group',
    y='freq',
    column='word',
    color='group'
)

alt.Chart(...)

**Anyways...** I feel like we've gotten that out of our system. In reality, we need to perform a lot more data cleaning and data organization before such an analysis will yield what we want.

Now back to distributed computing.

## Wrapping up our warmup
There is so much more to Parallel then we can discuss here. 

For example, you can use Parallel to execute commands on multiple nodes: <a href="https://www.gnu.org/software/parallel/parallel_tutorial.html#Remote-execution">https://www.gnu.org/software/parallel/parallel_tutorial.html#Remote-execution</a>. 

Parallel is one of the most useful distributed computing tools at your disposal. If you have a command line program that would benefit from running in a distributed fashion. Do NOT rewrite it until you have considered running it this way.

## Questions
1. Describe an instance where you would reach for GNU parallel instead of more advanced and integrated solutions.


2. When solving a distributed computing problem, what doesn't GNU parallel do for you? What are going to have to implement in the lab?





In [20]:
# Don't forget to push!